In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import AlgoLoop
import StatisticFunc

plt.style.use('Solarize_Light2')                     # 绘图风格
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 字体雅黑
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False    # 处理负号

# 预处理

## 提取并处理数据

### 权益类资产换手率

In [ ]:
Turnovers = pd.read_excel(io="Raw/国内资产换手率.xlsx")                      # 读取数据
Turnovers = Turnovers.drop([0], axis=0)                                   # 删除多余行
Turnovers["日期"] = pd.DatetimeIndex(Turnovers["日期"])                    # 类型转换
Turnovers = Turnovers.set_index("日期")                                   # 设置索引

In [ ]:
Turnovers = Turnovers.drop(Turnovers.index[0], axis=0)                   # 日期对齐

### GC007利率

In [ ]:
Rates = pd.read_excel(io="Raw/GC007利率.xlsx")           #  读取数据
Rates = Rates.drop([0, 1, 2], axis=0)                   # 删除多余行
Rates.columns = ['日期', 'GC007']                        # 修改列名
Rates["日期"] = pd.DatetimeIndex(Rates["日期"])           # 类型转换
Rates = Rates.set_index("日期")                          # 设置索引

In [ ]:
Rates = Rates.drop(Rates.index[0], axis=0)                     # 日期对齐

### 十年国债收益率

In [ ]:
BondRates = pd.read_excel(io="Raw/中债国债到期收益率(中债)(日).xls")        # 读取数据
BondRates.columns = ['日期', '十年国债收益率']                             # 修改列名
BondRates = BondRates.drop([0, 3870, 3871], axis=0)                     # 删除多余行
BondRates["日期"] = pd.DatetimeIndex(BondRates["日期"])                   # 类型转换
BondRates = BondRates.set_index("日期")                                  # 设置索引

### 各资产收盘价

In [ ]:
Assets = pd.read_excel(io="Raw/国内资产收盘价.xlsx")
Assets = Assets.drop([0], axis=0)                                   # 删除多余行
Assets["日期"] = pd.DatetimeIndex(Assets["日期"])                    # 类型转换
Assets = Assets.set_index("日期")                                   # 设置索引

## 衍生数据生成

### 铜金价格比 & 铜油价格比

In [ ]:
Data = Assets.merge(BondRates, how='left', left_index=True, right_index=True)

In [ ]:
Data = Data.merge(Rates, how='left', left_index=True, right_index=True)

In [ ]:
Data['铜金价格比'] = Data['中信证券COMEX铜期货'] / Data['中信证券COMEX黄金期货']
Data['铜油价格比'] = Data['中信证券COMEX铜期货'] / Data['中信证券WTI原油期货']

### 十年国债价格指数

In [ ]:
Data['持有时长'] = (Data.index - Data.index[0]).days
Data['10年国债价格指数'] = 100 - (Data['十年国债收益率'] - 3.0) * 8.2 + 3.0 * Data['持有时长'] / 365.0
Data['10年国债价格指数'] = Data['10年国债价格指数'].fillna(method='ffill')

## 相关性测试

### 铜金价格比值与10年国债，信用债的相关性

In [ ]:
# 皮尔逊相关系数
np.corrcoef(Data['10年国债价格指数'].values.astype(np.float32), Data['铜金价格比'].values.astype(np.float32))

In [ ]:
# 皮尔逊相关系数
np.corrcoef(Data['信用债3-5AAA'].values.astype(np.float32), Data['铜金价格比'].values.astype(np.float32))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

axs[0].plot(Data.index, Data['10年国债价格指数'], label='10年国债价格指数')
axs[0].plot(Data.index, Data['信用债3-5AAA'], label='信用债3-5AAA')
axs[0].set_xlabel('time')
axs[0].set_ylabel('债券价格指数')
axs[0].grid(True)

axs[1].plot(Data.index, Data['铜金价格比'])
axs[1].set_xlabel('time')
axs[1].set_ylabel('铜金价格比')
axs[1].grid(True)

fig.tight_layout()
plt.show()

### 铜油价格比值与沪深300，中证500的相关性

In [ ]:
Tmp1 = Data[['沪深300', '中证500', '铜油价格比']].dropna(how='any')
Tmp1 = Tmp1[200:-100]

In [ ]:
# 皮尔逊相关系数
np.corrcoef(Tmp1['沪深300'].values.astype(np.float32), Tmp1['铜油价格比'].values.astype(np.float32))

In [ ]:
# 皮尔逊相关系数
np.corrcoef(Tmp1['中证500'].values.astype(np.float32), Tmp1['铜油价格比'].values.astype(np.float32))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

axs[0].plot(Tmp1.index, Tmp1['沪深300'], label='沪深300')
axs[0].plot(Tmp1.index, Tmp1['中证500'], label='中证500')
axs[0].set_xlabel('time')
axs[0].set_ylabel('股指')
axs[0].legend()
axs[0].grid(True)

axs[1].plot(Tmp1.index, Tmp1['铜油价格比'])
axs[1].set_xlabel('time')
axs[1].set_ylabel('铜油价格比')
axs[1].grid(True)

fig.tight_layout()
plt.show()

## 计算各资产收益率

### 日内损益

In [ ]:
# 利用自制10年国债价格指数，替换上证10年国债
Assets['上证10年国债'] = Data['10年国债价格指数'].values

In [ ]:
Returns = Assets.pct_change(axis=0)
Returns = Returns.dropna(axis=0, how='all')           # 删除无数据日

### 累计损益

In [ ]:
cumReturns = (1.0 + Returns)
cumReturns = cumReturns.fillna(1.0)                    # 填充空值
cumReturns = cumReturns.cumprod()                      # 计算各资产累计收益率

In [ ]:
cumReturns.plot(figsize=(16, 8))

# 风险平价 （普通权重）

**资产明细**
- 沪深300指数
- 中证500指数
- 三年期3A信用债全价指数
- 十年期国债全价指数(自制)
- COMEX铜
- COMEX黄金
- NYMEX原油
- 铁矿石期货

回溯时间：2008年初-至今

买卖单笔手续费：千1

## 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]           # 调仓周期

NetValueDF = pd.DataFrame(index=Returns.index)
DailyPnlDF = pd.DataFrame(index=Returns.index) 
DrawDownDF = pd.DataFrame(index=Returns.index)

for dtime in timeWindow:
    tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, Returns, cumReturns, Turnovers, mode='plain', dt=dtime)
    
    StatisticFunc.WeightPlot(tradeDF, weightDF, '普通权重' + str(dtime))
    
    NetValueDF[str(dtime)+'组合净值'] = tradeDF['投资组合净值'] / 10000  
    DailyPnlDF[str(dtime)+'组合日收益率'] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
    DrawDownDF[str(dtime)+'最大回撤'] = tradeDF['最大回撤']   


## 回测表现汇总

In [ ]:
# 汇总
summaryDF = pd.DataFrame(index=['每月调仓', '每季度调仓', '每半年调仓', 
                                '每三季度调仓', '每年调仓'])

summaryDF["年平均收益率"] = DailyPnlDF.mean(axis=0).values * 250
summaryDF["年平均标准差"] = DailyPnlDF.std(axis=0).values * np.sqrt(250)
summaryDF["无基准夏普比率"] = summaryDF["年平均收益率"] / summaryDF["年平均标准差"]
summaryDF["最大回撤"] = DrawDownDF.min(axis=0).values
summaryDF

## 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

for col in NetValueDF.columns:
    plt.plot(NetValueDF.index, NetValueDF[col], lw=1.0, label=col)

plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线')
plt.show()

## 年间收益率分布

In [ ]:
yearReturns = StatisticFunc.annualReturns(NetValueDF)

In [ ]:
StatisticFunc.ReturnDist(yearReturns, 2)

## 月间收益率分布

In [ ]:
monthlyReturns = StatisticFunc.monthlyReturns(NetValueDF)

In [ ]:
StatisticFunc.ReturnDist(monthlyReturns, 2)

# 风险平价 + 杠杆策略（普通权重）

## 杠杆调整后的债券净值

In [ ]:
lever = 2.0                                 # 杠杆
leverReturns = Returns.copy()               # 创建副本 
leverCumReturns = cumReturns.copy()   

# 更新杠杆调整后的债券净值
leverReturns.loc[:, ['上证10年国债', '信用债3-5AAA']] = Returns.loc[:, ['上证10年国债', '信用债3-5AAA']] * (1.0 + lever) - Rates.values * lever / (365.0 * 100)

# 更新累计净值
leverCumReturns = (1.0 + leverReturns)
leverCumReturns = leverCumReturns.fillna(1.0)                    # 填充空值
leverCumReturns = leverCumReturns.cumprod()                      # 计算各资产累计收益率

In [ ]:
leverReturns

## 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]           # 调仓周期

leverNetValueDF = pd.DataFrame(index=Returns.index)
leverDailyPnlDF = pd.DataFrame(index=Returns.index) 
leverDrawDownDF = pd.DataFrame(index=Returns.index)

for dtime in timeWindow:
    tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='plain', dt=dtime)
    StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆普通权重'+str(dtime))
    
    leverNetValueDF[str(dtime)+'组合净值'] = tradeDF['投资组合净值'] / 10000
    leverDailyPnlDF[str(dtime)+'组合日收益率'] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
    leverDrawDownDF[str(dtime)+'最大回撤'] = tradeDF['最大回撤']   


## 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF = pd.DataFrame(index=['每月调仓', '每季度调仓', '每半年调仓', 
                                '每三季度调仓', '每年调仓'])

leverSummaryDF["年平均收益率"] = leverDailyPnlDF.mean(axis=0).values * 250
leverSummaryDF["年平均标准差"] = leverDailyPnlDF.std(axis=0).values * np.sqrt(250)
leverSummaryDF["无基准夏普比率"] = leverSummaryDF["年平均收益率"] / leverSummaryDF["年平均标准差"]
leverSummaryDF["最大回撤"] = leverDrawDownDF.min(axis=0).values
leverSummaryDF

## 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

for col in leverNetValueDF.columns:
    plt.plot(leverNetValueDF.index, leverNetValueDF[col], lw=1.0, label=col)

# plt.plot(cumReturns.index, cumReturns['中证500'], lw=1.5, label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 6.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆）')
plt.show()

## 年间收益率分布

In [ ]:
yearReturns = StatisticFunc.annualReturns(NetValueDF)

In [ ]:
StatisticFunc.ReturnDist(yearReturns, 2)

## 月间收益率分布

In [ ]:
monthlyReturns = StatisticFunc.monthlyReturns(NetValueDF)

In [ ]:
StatisticFunc.ReturnDist(monthlyReturns, 2)

# 风险平价 + 杠杆 + 因子策略（普通权重）

## 动量因子（横向比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]        # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_momentumX = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_momentumX = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_momentumX = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':False, 
           'reverseX':False, 'reverseT':False, 
           'turnover':False,  
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds,
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆横截面动量（裸权重）'+str(t+u))

        leverNetValueDF_momentumX.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000
        leverDailyPnlDF_momentumX.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_momentumX.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_momentumX = pd.DataFrame(index=multiIDX)

leverSummaryDF_momentumX["年平均收益率"] = leverDailyPnlDF_momentumX.mean(axis=0).values * 250
leverSummaryDF_momentumX["年平均标准差"] = leverDailyPnlDF_momentumX.std(axis=0).values * np.sqrt(250)
leverSummaryDF_momentumX["无基准夏普比率"] = leverSummaryDF_momentumX["年平均收益率"] / leverSummaryDF_momentumX["年平均标准差"]
leverSummaryDF_momentumX["最大回撤"] = leverDrawDownDF_momentumX.min(axis=0).values

leverSummaryDF_momentumX

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_momentumX.columns:
    plt.plot(leverNetValueDF_momentumX.index, leverNetValueDF_momentumX[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.5, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆横截面动量因子（裸权重）')
plt.show()

## 动量因子（时序比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_momentumT = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_momentumT = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_momentumT = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':True, 
           'reverseX':False, 'reverseT':False, 
           'turnover':False,  
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆时序动量（裸权重）'+str(t+u))

        leverNetValueDF_momentumT.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_momentumT.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_momentumT.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_momentumT = pd.DataFrame(index=multiIDX)

leverSummaryDF_momentumT["年平均收益率"] = leverDailyPnlDF_momentumT.mean(axis=0).values * 250
leverSummaryDF_momentumT["年平均标准差"] = leverDailyPnlDF_momentumT.std(axis=0).values * np.sqrt(250)
leverSummaryDF_momentumT["无基准夏普比率"] = leverSummaryDF_momentumT["年平均收益率"] / leverSummaryDF_momentumT["年平均标准差"]
leverSummaryDF_momentumT["最大回撤"] = leverDrawDownDF_momentumT.min(axis=0).values

leverSummaryDF_momentumT

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_momentumT.columns:
    plt.plot(leverNetValueDF_momentumT.index, leverNetValueDF_momentumT[col], label=col)

plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')
    
plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆时间序列动量因子（裸权重）')
plt.show()

## 反转因子（横向比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_reverseX = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_reverseX = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_reverseX = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
           'reverseX':True, 'reverseT':False, 
           'turnover':False, 
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆横截面反转（裸权重）'+str(t+u))

        leverNetValueDF_reverseX.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_reverseX.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_reverseX.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_reverseX = pd.DataFrame(index=multiIDX)

leverSummaryDF_reverseX["年平均收益率"] = leverDailyPnlDF_reverseX.mean(axis=0).values * 250
leverSummaryDF_reverseX["年平均标准差"] = leverDailyPnlDF_reverseX.std(axis=0).values * np.sqrt(250)
leverSummaryDF_reverseX["无基准夏普比率"] = leverSummaryDF_reverseX["年平均收益率"] / leverSummaryDF_reverseX["年平均标准差"]
leverSummaryDF_reverseX["最大回撤"] = leverDrawDownDF_reverseX.min(axis=0).values

leverSummaryDF_reverseX

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_reverseX.columns:
    plt.plot(leverNetValueDF_reverseX.index, leverNetValueDF_reverseX[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆横截面反转因子（裸权重）')
plt.show()

## 反转因子 （时序比较）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_reverseT = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_reverseT = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_reverseT = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
           'reverseX':False, 'reverseT':True, 
           'turnover':False, 
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆时序反转（裸权重）'+str(t+u))

        leverNetValueDF_reverseT.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_reverseT.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_reverseT.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_reverseT = pd.DataFrame(index=multiIDX)

leverSummaryDF_reverseT["年平均收益率"] = leverDailyPnlDF_reverseT.mean(axis=0).values * 250
leverSummaryDF_reverseT["年平均标准差"] = leverDailyPnlDF_reverseT.std(axis=0).values * np.sqrt(250)
leverSummaryDF_reverseT["无基准夏普比率"] = leverSummaryDF_reverseT["年平均收益率"] / leverSummaryDF_reverseT["年平均标准差"]
leverSummaryDF_reverseT["最大回撤"] = leverDrawDownDF_reverseT.min(axis=0).values

leverSummaryDF_reverseT

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_reverseT.columns:
    plt.plot(leverNetValueDF_reverseT.index, leverNetValueDF_reverseT[col], label=col)

plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')
    
plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆时序反转因子（裸权重）')
plt.show()

## 情绪因子（股指换手率）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_turnover = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_turnover = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_turnover = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
           'reverseX':False, 'reverseT':False, 
           'turnover':True, 
           'copperGold':False, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆换手率（裸权重）'+str(t+u))

        leverNetValueDF_turnover.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_turnover.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_turnover.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_turnover = pd.DataFrame(index=multiIDX)

leverSummaryDF_turnover["年平均收益率"] = leverDailyPnlDF_turnover.mean(axis=0).values * 250
leverSummaryDF_turnover["年平均标准差"] = leverDailyPnlDF_turnover.std(axis=0).values * np.sqrt(250)
leverSummaryDF_turnover["无基准夏普比率"] = leverSummaryDF_turnover["年平均收益率"] / leverSummaryDF_turnover["年平均标准差"]
leverSummaryDF_turnover["最大回撤"] = leverDrawDownDF_turnover.min(axis=0).values

leverSummaryDF_turnover

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_turnover.columns:
    plt.plot(leverNetValueDF_turnover.index, leverNetValueDF_turnover[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆股指换手率因子（裸权重）')
plt.show()

## 铜金价格比因子（十年国债反转）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_copperGold = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_copperGold = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_copperGold = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 
            'copperGold':True, 'copperGas':False}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆铜金价格比（裸权重）'+str(t+u))

        leverNetValueDF_copperGold.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_copperGold.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_copperGold.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_copperGold = pd.DataFrame(index=multiIDX)

leverSummaryDF_copperGold["年平均收益率"] = leverDailyPnlDF_copperGold.mean(axis=0).values * 250
leverSummaryDF_copperGold["年平均标准差"] = leverDailyPnlDF_copperGold.std(axis=0).values * np.sqrt(250)
leverSummaryDF_copperGold["无基准夏普比率"] = leverSummaryDF_copperGold["年平均收益率"] / leverSummaryDF_copperGold["年平均标准差"]
leverSummaryDF_copperGold["最大回撤"] = leverDrawDownDF_copperGold.min(axis=0).values

leverSummaryDF_copperGold

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_copperGold.columns:
    plt.plot(leverNetValueDF_copperGold.index, leverNetValueDF_copperGold[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆铜金价格比因子（裸权重）')
plt.show()

## 铜油价格比因子（沪深300动量）

### 程序运行

In [ ]:
# 运行
timeWindow = [20, 60, 120, 180, 240]        # 调仓周期
ups = [0.50, 1.00]                          # 上调幅度

multiIDX = pd.MultiIndex.from_product([timeWindow, ups], names=['交易周期', '上调幅度'])
leverNetValueDF_copperGas = pd.DataFrame(index=Returns.index, columns=multiIDX)
leverDailyPnlDF_copperGas = pd.DataFrame(index=Returns.index, columns=multiIDX) 
leverDrawDownDF_copperGas = pd.DataFrame(index=Returns.index, columns=multiIDX)

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 
            'copperGold':False, 'copperGas':True}

for t in timeWindow:
    for u in ups:
        tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, 
                                               mode='plain', dt=t, up=u, thresholds=thrds, 
                                               factorDict=factors)
        StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆铜油价格比（裸权重）'+str(t+u))

        leverNetValueDF_copperGas.loc[:, (t, u)] = tradeDF['投资组合净值'] / 10000  
        leverDailyPnlDF_copperGas.loc[:, (t, u)] = tradeDF['投资组合净值'] / tradeDF['投资组合净值'].shift(1) - 1.0
        leverDrawDownDF_copperGas.loc[:, (t, u)] = tradeDF['最大回撤']                      

### 回测表现汇总

In [ ]:
# 汇总
leverSummaryDF_copperGas = pd.DataFrame(index=multiIDX)

leverSummaryDF_copperGas["年平均收益率"] = leverDailyPnlDF_copperGas.mean(axis=0).values * 250
leverSummaryDF_copperGas["年平均标准差"] = leverDailyPnlDF_copperGas.std(axis=0).values * np.sqrt(250)
leverSummaryDF_copperGas["无基准夏普比率"] = leverSummaryDF_copperGas["年平均收益率"] / leverSummaryDF_copperGas["年平均标准差"]
leverSummaryDF_copperGas["最大回撤"] = leverDrawDownDF_copperGas.min(axis=0).values

leverSummaryDF_copperGas

### 各投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8), dpi=200)

for col in leverNetValueDF_copperGas.columns:
    plt.plot(leverNetValueDF_copperGas.index, leverNetValueDF_copperGas[col], label=col)
    
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], lw=3.0, label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('杠杆铜油价格比因子（裸权重）')
plt.show()